In [ ]:
import sys
import pyVoxelFEM
import MeshFEM, mesh
from tri_mesh_viewer import QuadHexViewer

sys.path.append('./helpers')
from ipopt_helpers import initializeTensorProductSimulator, problemObjectWrapper, initializeIpoptProblem

import numpy as np
import matplotlib.pyplot as plt
import ipopt
import time

# matplotlib default
FONTSIZE = 12
FIGSIZE = (9, 6)
plt.rcParams.update({'font.size': FONTSIZE})
plt.rcParams.update({"figure.figsize": FIGSIZE})

# Optimize in batches
Define optimization batches by their number of iterations. In each batch, filters can be reinitialized modifying their parameters or even adding or removing them from the TO problem.

In [ ]:
# 2D or 3D?
DIM = 3

# Material and "boundary conditions" (can be imposed also inside the domain) are read from file
MATERIAL_PATH = '../examples/materials/B9Creator.material'
if DIM == 2:
    BC_PATH = '../examples/bcs/cantilever_flexion_E.bc'
elif DIM == 3:
    BC_PATH = '../examples/bcs/3D/cantilever_flexion_SE.bc'

# TensorProductSimulator
if DIM == 2:
    orderFEM = [1, 1]
    domainCorners = [[0, 0], [2, 1]]
    gridDimensions = [60,, 30]
elif DIM == 3:
    orderFEM = [1, 1, 1]
    domainCorners = [[0, 0, 0], [3, 1, 1]]
    gridDimensions = [24, 8, 8]
elementsNumber = int(np.prod(gridDimensions))
uniformDensity = 1.0
E0 = 1
Emin = 1e-9
SIMPExponent = 3

# Constraints
maxVolume = 0.4
constraints = [pyVoxelFEM.TotalVolumeConstraint(maxVolume)]

# Topology Optimization problem
uniformDensity = maxVolume

In [ ]:
# Optimization batches setup
itersBatches = np.array([         20,     10,     10,     20])
smoothingBatches = np.array([      1,      2,      3,      1])
projectionBatches = np.array([ False,      1,      2,      4])
langelaarBatches = np.array([   True,   True,   True,   True])
totalIters = np.sum(itersBatches)
if not (itersBatches.size == smoothingBatches.size == projectionBatches.size == langelaarBatches.size):
    raise ValueError("All the batches must have the same size.")

# Simulator and objective initialization
tps = initializeTensorProductSimulator(
    orderFEM, domainCorners, gridDimensions, uniformDensity, E0, Emin, SIMPExponent, MATERIAL_PATH, BC_PATH
)
objective = pyVoxelFEM.ComplianceObjective(tps)
x0 = tps.getDensities()
history = []

# Optimization loop
pyVoxelFEM.benchmark_reset() # profile
for batch in range(len(itersBatches)):
    # Filters initialization
    filters = []
    if smoothingBatches[batch] != False:
        filters.append(pyVoxelFEM.SmoothingFilter())
    if projectionBatches[batch] != False:
        filters.append(pyVoxelFEM.ProjectionFilter())
    if langelaarBatches[batch] != False:
        filters.append(pyVoxelFEM.LangelaarFilter())
    # Problem initialization
    top = pyVoxelFEM.TopologyOptimizationProblem(tps, objective, constraints, filters)
    nonLinearProblem, history = initializeIpoptProblem(top, history)
    nonLinearProblem.addOption('tol', 1e-16)
    # Filters update
    for filt in filters:
        if type(filt) == pyVoxelFEM.SmoothingFilter:
            filt.radius = smoothingBatches[batch]
        elif type(filt) == pyVoxelFEM.ProjectionFilter:
            filt.beta = projectionBatches[batch]
    # Optimize
    nonLinearProblem.addOption('max_iter', int(itersBatches[batch])-1)
    x0, info = nonLinearProblem.solve(x0)
pyVoxelFEM.benchmark_report()

In [ ]:
view = QuadHexViewer(*tps.getMesh(), scalarField=np.ones(x0.shape))
view.show()

In [ ]:
view.update(scalarField=history.density[-1])

In [ ]:
for density in history.density:
    view.update(scalarField=density)
    time.sleep(0.05)

### Plot results
Run first cell below to create a figure. The figure can be updated with new results, keeping the previously plotted lines. 

In [ ]:
# Create new figure
fig, axes = plt.subplots(2, 1, figsize=(FIGSIZE[0], FIGSIZE[1]*1.2))
plt.close()

In [ ]:
# Update existing figure
plotLegend = ''
history.plotNondiscreteness(axes[0], plotLegend)
history.plotObjective(axes[1])
axes[0].legend()
plt.close()

In [ ]:
# Show figure
fig

In [ ]:
# Save figure
plt.savefig(ADD_NAME_HERE + '.png', format='png', dpi=600)

In [ ]:
# Delete figure
if ("fig" in locals()) and ("axes" in locals()):
    del fig, axes

### Export result to `.msh`

In [ ]:
# Export final density to .msh
mfw = mesh.MSHFieldWriter(ADD_PATH_HERE + '.msh', *tps.getMesh())
mfw.addField('density', history.density[-1])

In [ ]:
# Export whole history to multiple .msh files (for animation recording)
for iteration, density in enumerate(history.density):
    mfw = mesh.MSHFieldWriter(ADD_PATH_HERE + 'iter' + str(iteration).zfill(4) + '.msh', *tps.getMesh())
    mfw.addField('density', density)